In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#import lightgbm as lgb
#import xgboost as xgb
from sklearn.metrics import roc_auc_score
#import xgboost as xgb

In [48]:
#train=pd.read_csv("./iiitb2019nyctaxifare/TrainTest/train.csv",nrows=3000000)
#print("Shape of Training Data",train.shape)
test=pd.read_csv("./iiitb2019nyctaxifare/TrainTest/test.csv")
print("Shape of Testing Data", test.shape)

Shape of Testing Data (11084772, 7)


In [49]:
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]
def clean_data(data):
    boundary={'min_lng':-74.263242,
              'min_lat':40.573143,
              'max_lng':-72.986532, 
              'max_lat':41.709555}
    
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
   # if 'fare_amount' in data.columns:
    #    data=data[data['fare_amount']>=0]
     #   data.loc[~((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
    #        (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
     #       (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
     #       (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=1
      #  data.loc[((data.pickup_longitude >= boundary['min_lng'] ) & (data.pickup_longitude <= boundary['max_lng']) &
       #     (data.pickup_latitude >= boundary['min_lat']) & (data.pickup_latitude <= boundary['max_lat']) &
        #    (data.dropoff_longitude >= boundary['min_lng']) & (data.dropoff_longitude <= boundary['max_lng']) &
         #   (data.dropoff_latitude >=boundary['min_lat']) & (data.dropoff_latitude <= boundary['max_lat'])),'is_outlier_loc']=0

    #print("Outlier vs Non Outlier Counts")
    #print(data['is_outlier_loc'].value_counts())

    # Let us drop rows, where location is outlier
        #data=data.loc[data['is_outlier_loc']==0]
        #data.drop(['is_outlier_loc'],axis=1,inplace=True)
        
    #data=data[data['passenger_count']<=8]
    data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
    return data

In [50]:
#train=clean_data(train)
#print("Shape of Training Data after cleaning ",train.shape)
test=clean_data(test)
print("Shape of Testing Data after cleaning", test.shape)

Shape of Testing Data after cleaning (11084772, 12)


In [8]:
def processDataForModelling(data,target,drop_cols,is_train=True,split=0.25):
    data_1=data.drop(drop_cols,axis=1)
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

In [9]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=True,split=0.2)

Shape of Training Features (2348335, 10)
Shape of Validation Features  (587084, 10)


In [10]:
test_data=processDataForModelling(test,'fare_amount',drop_cols=['key','pickup_datetime'],is_train=False)

Shape of Test Data (11084757, 10)


In [11]:
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred=np.round(lm.predict(X_test),2)
lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
print("RMSE for Linear Regression is ",lm_rmse)

RMSE for Linear Regression is  8.211724468146713


In [56]:
key_value = test['key']
key_value.shape

(11084772,)

In [59]:
test = test.drop(columns = ['key'])

In [60]:
test.columns

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year'],
      dtype='object')

In [62]:
test.fillna(method = 'ffill',inplace = True)

In [63]:
## Now Linear regression on test
lr_pred=lm.predict(test)

In [64]:
pred_lr = pd.DataFrame(lr_pred)

In [65]:
datasets_lr = pd.concat([key_value,pred_lr[0]], axis = 1)

In [66]:
datasets_lr.shape

(11084772, 2)

In [67]:
datasets_lr.columns = ['key','fare_amount']

In [68]:
datasets_lr.isnull().sum(axis = 0)

key            0
fare_amount    0
dtype: int64

In [41]:
datasets_lr.fillna(1,inplace = True)

In [42]:
datasets_lr.isnull().sum(axis = 0)

key            0
fare_amount    0
dtype: int64

In [69]:
datasets_lr.to_csv('sub_lr4.csv',index=False)

In [13]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 883,n_jobs=-1)
rf.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
rf_pred= rf.predict(X_test)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, y_test))
print("RMSE for Random Forest is ",rf_rmse)